<h5>From git <a href='https://github.com/CivilNet/Gemfield/blob/master/src/python/pascal_voc_xml2json/pascal_voc_xml2json.py'>repository</a> vocxmltococo format works only for bbox with in the format(xmin, xmax, ymin, ymax)</h5>

In [ ]:
import xml.etree.ElementTree as ET
import os
import json

coco = dict()
coco['images'] = []
coco['type'] = 'instances'
coco['annotations'] = []
coco['categories'] = []

category_set = dict()
image_set = set()

category_item_id = 0
image_id = 0
annotation_id = 0

def addCatItem(name):
    global category_item_id
    category_item = dict()
    category_item['supercategory'] = 'none'
    category_item_id += 1
    category_item['id'] = category_item_id
    category_item['name'] = name
    coco['categories'].append(category_item)
    category_set[name] = category_item_id
    return category_item_id

def addImgItem(file_name, size):
    global image_id
    if file_name is None:
        raise Exception('Could not find filename tag in xml file.')
    if size['width'] is None:
        raise Exception('Could not find width tag in xml file.')
    if size['height'] is None:
        raise Exception('Could not find height tag in xml file.')
    image_id += 1
    image_item = dict()
    image_item['id'] = image_id
    image_item['file_name'] = file_name
    image_item['width'] = size['width']
    image_item['height'] = size['height']
    coco['images'].append(image_item)
    image_set.add(file_name)
    return image_id

def addAnnoItem(object_name, image_id, category_id, bbox):
    global annotation_id
    annotation_item = dict()
    annotation_item['segmentation'] = []
    seg = []
    #bbox[] is x,y,w,h
    #left_top
    seg.append(bbox[0])
    seg.append(bbox[1])
    #left_bottom
    seg.append(bbox[0])
    seg.append(bbox[1] + bbox[3])
    #right_bottom
    seg.append(bbox[0] + bbox[2])
    seg.append(bbox[1] + bbox[3])
    #right_top
    seg.append(bbox[0] + bbox[2])
    seg.append(bbox[1])

    annotation_item['segmentation'].append(seg)

    annotation_item['area'] = bbox[2] * bbox[3]
    annotation_item['iscrowd'] = 0
    annotation_item['ignore'] = 0
    annotation_item['image_id'] = image_id
    annotation_item['bbox'] = bbox
    annotation_item['category_id'] = category_id
    annotation_id += 1
    annotation_item['id'] = annotation_id
    coco['annotations'].append(annotation_item)

def parseXmlFiles(xml_path): 
    for f in os.listdir(xml_path):
        if not f.endswith('.xml'):
            continue
        
        bndbox = dict()
        size = dict()
        current_image_id = None
        current_category_id = None
        file_name = None
        size['width'] = None
        size['height'] = None
        size['depth'] = None

        xml_file = os.path.join(xml_path, f)
        print(xml_file)

        tree = ET.parse(xml_file)
        root = tree.getroot()
        if root.tag != 'annotation':
            raise Exception('pascal voc xml root element should be annotation, rather than {}'.format(root.tag))

        #elem is <folder>, <filename>, <size>, <object>
        for elem in root:
            current_parent = elem.tag
            current_sub = None
            object_name = None
            
            if elem.tag == 'folder':
                continue
            
            if elem.tag == 'filename':
                file_name = elem.text
                if file_name in category_set:
                    raise Exception('file_name duplicated')
                
            #add img item only after parse <size> tag
            elif current_image_id is None and file_name is not None and size['width'] is not None:
                if file_name not in image_set:
                    current_image_id = addImgItem(file_name, size)
                    print('add image with {} and {}'.format(file_name, size))
                else:
                    raise Exception('duplicated image: {}'.format(file_name)) 
            #subelem is <width>, <height>, <depth>, <name>, <bndbox>
            for subelem in elem:
                bndbox ['xmin'] = None
                bndbox ['xmax'] = None
                bndbox ['ymin'] = None
                bndbox ['ymax'] = None
                
                current_sub = subelem.tag
                if current_parent == 'object' and subelem.tag == 'name':
                    object_name = subelem.text
                    if object_name not in category_set:
                        current_category_id = addCatItem(object_name)
                    else:
                        current_category_id = category_set[object_name]

                elif current_parent == 'size':
                    if size[subelem.tag] is not None:
                        raise Exception('xml structure broken at size tag.')
                    size[subelem.tag] = int(subelem.text)

                #option is <xmin>, <ymin>, <xmax>, <ymax>, when subelem is <bndbox>
                for option in subelem:
                    if current_sub == 'bndbox':
                        if bndbox[option.tag] is not None:
                            raise Exception('xml structure corrupted at bndbox tag.')
                        bndbox[option.tag] = int(option.text)

                #only after parse the <object> tag
                if bndbox['xmin'] is not None:
                    if object_name is None:
                        raise Exception('xml structure broken at bndbox tag')
                    if current_image_id is None:
                        raise Exception('xml structure broken at bndbox tag')
                    if current_category_id is None:
                        raise Exception('xml structure broken at bndbox tag')
                    bbox = []
                    #x
                    bbox.append(bndbox['xmin'])
                    #y
                    bbox.append(bndbox['ymin'])
                    #w
                    bbox.append(bndbox['xmax'] - bndbox['xmin'])
                    #h
                    bbox.append(bndbox['ymax'] - bndbox['ymin'])
                    print('add annotation with {},{},{},{}'.format(object_name, current_image_id, current_category_id, bbox))
                    addAnnoItem(object_name, current_image_id, current_category_id, bbox )

if __name__ == '__main__':
    xml_path = 'Annotations'
    json_file = 'instances.json'
    parseXmlFiles(xml_path)
    json.dump(coco, open(json_file, 'w'))

<h5>Converting COCO Json to DataFrame</h5>

In [7]:
import json
import pandas as pd
import numpy as np
import shapely.geometry as sg
import matplotlib.pyplot as plt

target_file = rf"D:\Projects\Ship_Campaign\ShipRSImageNet_V1\ShipRSImageNet_V1\COCO_Format\ShipRSImageNet_bbox_val_level_3.json"

with open(target_file, 'r') as file:
    js_file = json.loads(file.read())
    print("Key Pairs:", ", ".join(js_file.keys()))
    print("Images Present:", len(js_file['images']))
    print("Categories Present:", len(js_file['categories']))
    print("Annotations Present:", len(js_file['annotations']))

image_dict = js_file['images']
cat_dict = js_file['categories']
anno_dict = js_file['annotations']

img_df = pd.DataFrame(image_dict)
cat_df = pd.DataFrame(cat_dict)
anno_df = pd.DataFrame(anno_dict)

cat_df = cat_df.rename(columns={'id':'category_id', 'color':'cat_color'})
main_df = pd.merge(anno_df, cat_df, on='category_id')

img_df = img_df.rename(columns={'id':'image_id'})
main_df = pd.merge(main_df, img_df, on='image_id')
display(main_df.head())
main_df.to_csv('val_lvl_3.csv', index=False)

Key Pairs: images, categories, annotations
Images Present: 550
Categories Present: 50
Annotations Present: 3103


,area,category_id,ignore,segmentation,iscrowd,bbox,image_id,id,name,supercategory,file_name,width,height
0,27360.0,40,0,"[[120.0, 200.0, 76.0, 260.0, 364.0, 474.0, 408...",0,"[76, 200, 333, 275]",1,1,Cargo,Cargo,000746.bmp,930,930
1,80949.0,47,0,"[[373.0, 96.0, 305.0, 194.0, 853.0, 576.0, 921...",0,"[305, 96, 617, 481]",1,2,Oil Tanker,Oil Tanker,000746.bmp,930,930
2,44690.0,40,0,"[[476.0, 647.0, 552.0, 674.0, 739.0, 163.0, 66...",0,"[476, 136, 264, 539]",12,358,Cargo,Cargo,100000829.bmp,1160,824
3,54513.0,40,0,"[[151.0, 368.0, 186.0, 440.0, 789.0, 143.0, 75...",0,"[151, 71, 639, 370]",16,366,Cargo,Cargo,100000866.bmp,1179,825
4,57290.0,40,0,"[[268.0, 672.0, 305.0, 748.0, 912.0, 456.0, 87...",0,"[268, 380, 645, 369]",16,367,Cargo,Cargo,100000866.bmp,1179,825


In [3]:
# CSV or DataFrame to Coco Converter
# Need to change some code for "anno_id", "bbox"+2C and "area"

import json
import pandas as pd
import numpy as np
from ast import literal_eval

df = pd.read_csv('ship_train_val_lvl3.csv', converters={'segmentation' : literal_eval, 'bbox' : literal_eval})
df.columns = map(str.lower, df.columns)
df.columns
try:
    df = df.drop(columns=['image_id', 'image_path', 'created_at', 'updated_at',
                      'inner_id', 'type_id', 'time_taken', 'no_of_annotations', 'temp_cat',
                      'no_of_unique_objects', 'unique_objects', 'objects_with_count',
                      'average_time_per_object', 'labels_created_at', 'labels_updated_at', 'annotation_id', 'label'])
except Exception:
    pass

images = []
categories = []
annotations = []
final_dict = {}

df['image_id'] = df['file_name'].astype('category').cat.codes
df['categoryid'] = pd.Categorical(df['name'], ordered=True).codes
df['categoryid'] = df['categoryid']+1
df['anno_id'] = df.index

def image(row):
    image = {
        "file_name" : row.file_name,
        "id" : row.image_id,
        "width" : row.width,
        "height" : row.height,
    }
    return image

def category(row):
    category = {
        "id" : row.categoryid,
        "name" : row.name,
        "supercategory" : ""#row.supercategory
    }
    return category

def annotation(row):
    annotation = {
        "id" : row.anno_id,
        "image_id" : row.image_id,
        "category_id" : row.categoryid,
        "segmentation" : row.segmentation,
        "bbox" : row.bbox,
        "iscrowd" : 0,
        "area" : row.area,
    }
    return annotation

img_df = df.drop_duplicates(subset=['image_id']).sort_values(by='image_id', ignore_index=True)
for row in img_df.itertuples():
    images.append(image(row))

cat_df = df.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid', ignore_index=True)
for row in cat_df.itertuples():
    categories.append(category(row))

for row in df.itertuples():
    annotations.append(annotation(row))

final_dict['images'] = images
final_dict['categories'] = categories
final_dict['annotations'] = annotations

with open('ship_train_val_lvl_3.json', 'w') as output:
    wr_dict = json.dump(final_dict, output)